<a href="https://colab.research.google.com/github/monya-9/deep-learning-practice/blob/main/09_BERT_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사전학습된 BERT에 fine-tuning 해보기 (감정 분석 등)

In [ ]:
# 1. 라이브러리 설치
# 기존 패키지를 강제로 삭제합니다.
!pip uninstall -y transformers datasets evaluate

# 캐시를 무시하고 최신 버전을 다시 설치합니다.
!pip install --no-cache-dir --upgrade transformers datasets evaluate

Found existing installation: transformers 4.55.2
Uninstalling transformers-4.55.2:
  Successfully uninstalled transformers-4.55.2
Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 251.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 346.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 330.7 MB/s eta 0:00:00


In [ ]:
# 2. 라이브러리 불러오기
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import torch

In [ ]:
# 3. 데이터셋 로드 (IMDb 영화 리뷰 감정 분석) 및 토크나이
dataset = load_dataset("imdb")

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

- IMDb 영화 리뷰 데이터셋 로드
- AutoTokenizer로 BERT 토크나이저 로드 (bert-base-uncased 사용, 대소문자 구분x)
- 각 리뷰를 토큰화(tokenization) 하고, 최대 길이 256으로 패딩/자르기 수행.
- dataset.map으로 전체 데이터셋에 적용

In [ ]:
# 3. 데이터셋 분리
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))  # 빠른 학습용 소규모
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))

- 학습 속도를 빠르게 하기 위해 작은 샘플로 학습/테스트셋 생성
- shuffle(seed=42)로 데이터 섞기, select(range(N))로 상위 N개 선택

In [ ]:
# 4. BERT 모델 로드 및 Fine-tuning
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# 평가 지표 (Accuracy)
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# 학습 설정
training_args = TrainingArguments(
    output_dir="test_trainer",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    push_to_hub=False,
    report_to="none"
)

# Trainer 준비
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 학습 실행
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4051845849.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.584300
100,0.355400
150,0.309700
200,0.328400
250,0.296600
300,0.315600
350,0.190700
400,0.152300
450,0.217700
500,0.211400


TrainOutput(global_step=626, training_loss=0.27120188630807895, metrics={'train_runtime': 501.8516, 'train_samples_per_second': 19.926, 'train_steps_per_second': 1.247, 'total_flos': 1315555276800000.0, 'train_loss': 0.27120188630807895, 'epoch': 2.0})

1. BERT 모델 로드
  - 사전 학습된 BERT 모델 불러오기
  - 감정 분석이므로 클래스 수를 2로 설정(Positive / Negative)
2. 평가 지표 정의
  - 정확도(Accuracy)를 평가 지표로 사용
  - 모델 출력 logits에서 argmax로 예측 라벨 추출 후, 실제 레이블과 비교
3. 학습 설정
  -  학습 관련 하이퍼파라미터 설정
    - 학습률 2e-5, 배치 사이즈 16, 2 epoch 학습
    - 가중치 감쇠(weight_decay) 적용
    - 로그 저장 디렉토리와 로깅 간격 지정
    - save_strategy="epoch": epoch마다 모델 저장
    - report_to="none": wandb 등 외부 기록 비활성화
4. Trainer 객체 생성
  - Hugging Face Trainer는 학습 루프, 평가, 체크포인트 저장 등을 자동으로 처리
  - 토크나이저와 평가 함수(compute_metrics)도 전달.
5. 학습 실행
  - BERT를 IMDb 샘플 데이터에 맞춰 Fine-tuning 수행

In [ ]:
# 10. 평가 실행 및 추론
# 입력 문장을 토크나이징
text = "The movie was fantastic! I really loved it." # Positive 😀
# text = "The movie was terrible and a complete waste of time." #Negative 😡
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# 모델과 같은 디바이스로 이동
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# 추론
with torch.no_grad():
    logits = model(**inputs).logits

pred = torch.argmax(logits, dim=-1).item()
print("입력 문장:", text)
print("예측 결과:", "Positive 😀" if pred == 1 else "Negative 😡")

입력 문장: The movie was fantastic! I really loved it.
예측 결과: Positive 😀


- 임의 문장 하나를 토크나이징 후, 모델 디바이스(CPU/GPU)로 이동
- torch.no_grad()로 추론 모드에서 연산
- logits에서 argmax로 예착 라벨 선택
- 결과를 Positive 😀 / Negative 😡로 출력